# Ejemplo: Recocido Simulado (RS)

## Problema del agente viajero

El problema del agente viajero (Travelling salesman problem) considera un conjunto finito de $n$ ciudades y la distancia entre cada par de ellas. El objetivo es encontrar el camino más corto que visite cada ciudad exactamente una vez y regrese a la ciudad de origen. Formalmente se define como sigue:

\begin{equation}
    \label{eq:TSP}
    \begin{array}{rll}
    \text{minimizar:} & f(x) = d(x_n, x_1) + \sum_{i=1}^{n-1} d(x_i, x_{i+1}) &  \\
    \text{tal que} & x_i \in \{1,2,\cdots,n\} & \\
    \end{array}
\end{equation}


donde $d(x_i, x_j)$ es la distancia de ir de la ciudad $x_i$ a la ciudad $x_j$, $n$ es el número de ciudades y $x$ es una permutación de las $n$ ciudades.

## Principales componentes de RS

Librerías de Python que vamos a utilizar.

In [81]:
import numpy
import math

Datos de entrada.

In [82]:
n = 5
dist_matrix = [
                [0,49,30,53,72],
                [49,0,19,38,32],
                [30,19,0,41,98],
                [53,38,41,0,52],
                [72,32,98,52,0],
               ]

### Representación de una solución

Cada solución es una permutación de las $n$ ciudades. En Python será una lista de $n$ elementos. Nombraremos a las ciudades como $0$, $1$, $\cdots$, $n-1$ para que coincidan con los índices de nuestra lista. El primer elemento siempre será $0$ porque es la ciudad de partida. 

In [83]:
x = [0,4,2,1,3]

### Solución inicial

Utilizaremos una estrategia voraz: Empezamos en la ciudad $0$, posteriormente revisamos las 4 ciudades restantes y elegimos la que tenga una distancia menor a la ciudad $0$. Repitemos el proceso hasta tener nuestra permutación.

In [84]:
def getNextCity(x, dist_matrix, n):
    current_city = x[-1]
    min_dist = max(dist_matrix[current_city])
    
    for i in range(1, n):
        if (i not in x) and (dist_matrix[current_city][i] < min_dist):
            min_city = i
            min_dist = dist_matrix[current_city][i]
    
    return min_city

def getInitialSolution(n, dist_matrix):
    x = [0]
    
    while len(x) != n:
        x.append(getNextCity(x, dist_matrix, n))
        
    return x    

In [85]:
x_0 = getInitialSolution(n, dist_matrix)
print ("x_0: ", x_0)

x_0:  [0, 2, 1, 4, 3]


### Función objetivo

In [86]:
def f(n, dist_matrix, x):
    cost = dist_matrix[x[-1]][0]
    
    for i in range(n-1):
        cost += dist_matrix[x[i]][x[i+1]]
        
    return cost

In [87]:
print("f(x)\t= ", f(n, dist_matrix, x))
print("f(x_0)\t= ", f(n, dist_matrix, x_0))

f(x)	=  280
f(x_0)	=  186


### Vecindario

El vecindario se genera eligiendo aleatoriamente una posición $i$ de la permutación. Posteriormente, se generan $N-2$ soluciones moviendo la ciudad que está en la posición $p$ a cualquiera de las otras posiciones posibles. En Python utilizaremos una lista de listas para almacenar el vecindario. Cada lista va a tener una permutación y su valor en la función objetivo. 

In [88]:
def getNeighborhood(x, n, dist_matrix):
    aux_x = x.copy()
    #Elegimos una posición aleatoria
    i = numpy.random.randint(1, len(x))
    print ("Posición aleatoria: ", i)
    city = aux_x.pop(i)
    
    neighborhood = []
    for j in range(1, len(x)):
        if j != i:
            new_sol = aux_x.copy()
            new_sol.insert(j, city)
            neighborhood.append([new_sol, f(n, dist_matrix, new_sol)])
            
    return neighborhood

In [90]:
print ("Solución actual: ", x_0)
neighborhood = getNeighborhood(x_0, n, dist_matrix)
print ("Vecindario:")
for e in neighborhood:
    print(e)

Solución actual:  [0, 2, 1, 4, 3]
Posición aleatoria:  4
Vecindario:
[[0, 3, 2, 1, 4], 217]
[[0, 2, 3, 1, 4], 213]
[[0, 2, 1, 3, 4], 211]


Dado que elegiremos de manera aleatoria una solución del vecindario, no es necesario generar todo el vecindario. Por lo tanto, crearemos una función que nos permita obtener una única solución.

In [91]:
def nextSolution(x, n, dist_matrix):
    aux_x = x.copy()
    #Elegimos una posición aleatoria
    i = numpy.random.randint(1, len(x))
    #print ("Posición aleatoria: ", i)
    city = aux_x.pop(i)
    
    #Elegimos una nueva posición
    j = numpy.random.randint(1, len(x))
    while i == j:
        j = numpy.random.randint(1, len(x))
    
    aux_x.insert(j, city)
            
    return aux_x, f(n, dist_matrix, aux_x)

### Temperatura

Para este ejemplo vamos a utilizar una función lineal para disminuir la temperatura.

In [92]:
def updateTemperature(t):
    return 0.9*t

In [93]:
t = 1000
print ("Temperatura: ", t)
new_t = updateTemperature(t)
print ("Temperatura: ", new_t)

Temperatura:  1000
Temperatura:  900.0


# Algoritmo completo de un RS simple

In [94]:
#Recocido simulado simple para el problema del agente viajero
def SimulatedAnnealing(t_0, t_f, n, dist_matrix):
    x_0 = getInitialSolution(n, dist_matrix)
    x = x_0
    fx = f(n, dist_matrix, x)
    t = t_0
    x_best = x.copy()
    f_best = fx
    
    print("Solución inicial: ", x, fx)
    while t >= t_f:
        new_x, new_f = nextSolution(x, n, dist_matrix)
        
        if new_f <= f_best:
            x_best = new_x.copy()
            f_best = new_f
        
        if new_f < fx or numpy.random.random() < math.exp(-1.0*(new_f-fx)/t):
            x = new_x
            fx = new_f
            
        t = updateTemperature(t)
        #print(x, fx)
        
    return x_best, f_best

Ejecutamos el algoritmo.  

In [99]:
n = 5
dist_matrix = \
[\
[0,49,30,53,72],\
[49,0,19,38,32],\
[30,19,0,41,98],\
[53,38,41,0,52],\
[72,32,98,52,0],\
]

x, fx = SimulatedAnnealing(10000, 0.1, n, dist_matrix)
print("Solución encontrada: ", x, fx)

Solución inicial:  [0, 2, 1, 4, 3] 186
Solución encontrada:  [0, 2, 1, 4, 3] 186


Ejecutamos para una instancia más grande del problema

In [102]:
n = 10
dist_matrix = \
[\
[0,49,30,53,72,19,76,87,45,48],\
[49,0,19,38,32,31,75,69,61,25],\
[30,19,0,41,98,56,6,6,45,53],\
[53,38,41,0,52,29,46,90,23,98],\
[72,32,98,52,0,63,90,69,50,82],\
[19,31,56,29,63,0,60,88,41,95],\
[76,75,6,46,90,60,0,61,92,10],\
[87,69,6,90,69,88,61,0,82,73],\
[45,61,45,23,50,41,92,82,0,5],\
[48,25,53,98,82,95,10,73,5,0],\
]

x, fx = SimulatedAnnealing(100000, 0.01, n, dist_matrix)
print("Solución encontrada ", x, fx)

Solución inicial:  [0, 5, 3, 8, 9, 6, 2, 7, 1, 4] 271
Solución encontrada  [0, 5, 3, 8, 9, 6, 2, 7, 1, 4] 271


Realizamos $m$ ejecuciones de nuestro algoritmo y obtenemos la corrida que encontró la mejor solución, la corrida que encontró la peor solución, la media y la desviación estándar de las $m$ corridas con respecto al valor de la función objetivo.

In [103]:
m=21
t_0 = 100000
t_f = 0.01
sol = []
for i in range(m):
    print("Ejecución ", i, ": ")
    x_best, f_best, = SimulatedAnnealing(t_0, t_f, n, dist_matrix)
    sol.append([f_best, x_best])
    print(x_best, f_best)
    
sol.sort()
print("*************** Mejor solución ***************")
print(sol[0][0], sol[0][1])

print("*************** Peor solución ****************")
print(sol[-1][0], sol[-1][1])

print("****************** Mediana *******************")
med = m//2
print(sol[med][0], sol[med][1])

f_sol = [x[0] for x in sol]
print("Media: ", numpy.mean(f_sol))
print("Desviación estándar: ", numpy.std(f_sol))

Ejecución  0 : 
Solución inicial:  [0, 5, 3, 8, 9, 6, 2, 7, 1, 4] 271
[0, 5, 3, 8, 9, 6, 2, 7, 1, 4] 271
Ejecución  1 : 
Solución inicial:  [0, 5, 3, 8, 9, 6, 2, 7, 1, 4] 271
[0, 5, 3, 8, 9, 6, 2, 7, 1, 4] 271
Ejecución  2 : 
Solución inicial:  [0, 5, 3, 8, 9, 6, 2, 7, 1, 4] 271
[0, 5, 3, 8, 9, 6, 2, 7, 1, 4] 271
Ejecución  3 : 
Solución inicial:  [0, 5, 3, 8, 9, 6, 2, 7, 1, 4] 271
[0, 5, 3, 8, 9, 6, 2, 7, 1, 4] 271
Ejecución  4 : 
Solución inicial:  [0, 5, 3, 8, 9, 6, 2, 7, 1, 4] 271
[0, 5, 3, 8, 9, 6, 2, 7, 1, 4] 271
Ejecución  5 : 
Solución inicial:  [0, 5, 3, 8, 9, 6, 2, 7, 1, 4] 271
[0, 5, 3, 8, 9, 6, 2, 7, 1, 4] 271
Ejecución  6 : 
Solución inicial:  [0, 5, 3, 8, 9, 6, 2, 7, 1, 4] 271
[0, 5, 3, 8, 9, 6, 2, 7, 1, 4] 271
Ejecución  7 : 
Solución inicial:  [0, 5, 3, 8, 9, 6, 2, 7, 1, 4] 271
[0, 5, 3, 8, 9, 6, 2, 7, 1, 4] 271
Ejecución  8 : 
Solución inicial:  [0, 5, 3, 8, 9, 6, 2, 7, 1, 4] 271
[0, 1, 4, 7, 2, 6, 9, 8, 3, 5] 248
Ejecución  9 : 
Solución inicial:  [0, 5, 3, 8, 9, 6, 2

Recordemos que esta es la versión más simple de un RS, se pueden hacer cambios en el diseño para mejorar la eficacia del algoritmo.